<a href="https://colab.research.google.com/github/jedrexd/WTUM_11_2022/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df = pd.read_csv("/content/WTUM_11_2022/train.csv", header=0)
df = df.iloc[:,1:]
df

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df['MSSubClass'] = LabelEncoder().fit_transform(df['MSSubClass'].astype('str'))
df['MSZoning'] = LabelEncoder().fit_transform(df['MSZoning'].astype('str'))
df.replace('LotFrontage', {"NA": 0})
df['LotFrontage'] = StandardScaler().fit_transform(np.array(df['LotFrontage']).reshape(-1, 1));
df['LotArea'] = StandardScaler().fit_transform(df['LotArea'].values.reshape(-1, 1))
df['Street'] = LabelEncoder().fit_transform(df['Street'].astype('str'))
df['Alley'] = LabelEncoder().fit_transform(df['Alley'].astype('str'))
lotShapeDict = {"Reg": 1, "IR1": 2, "IR2": 3, "IR3": 4}
df.replace('LotShape', lotShapeDict)
df['LandContour'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))
df['LotConfig'] = LabelEncoder().fit_transform(df['LotConfig'].astype('str'))
df['LandSlope'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))
df['Neighborhood'] = LabelEncoder().fit_transform(df['Neighborhood'].astype('str'))
conditionDict = {"Artery": 1, "Feedr": 2, "Norm": 3,	"RRNn": 4, "RRAn": 5, "PosN": 6, "PosA": 7, "RRNe": 8, "RRAe": 9}
df.replace('Condition1', conditionDict)
df.replace('Condition2', conditionDict)
utilitiesDict = {"AllPub": 3, "NoSewr": 2, "NoSeWa": 1, "ELO": 0}
df.replace('Utilities', utilitiesDict);
df['BldgType'] = LabelEncoder().fit_transform(df['BldgType'].astype('str'))
df['HouseStyle'] = LabelEncoder().fit_transform(df['HouseStyle'].astype('str'))

# YearBuilt, YearRemodAdd, OverallCond, OverallQual: TBD

In [ ]:
print(df['GarageCars'].isna().sum())

In [ ]:
print(df['GarageArea'].isna().sum())
df['GarageArea'] = StandardScaler().fit_transform(df['GarageArea'].values.reshape(-1,1))

In [ ]:
print(df['GarageQual'].isna().sum())
unq_vals = df['GarageQual'].unique().astype('str')
df['GarageQual'] = df['GarageQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0})

In [ ]:
print(df['GarageCond'].isna().sum())
print(df['GarageCond'].unique())
df['GarageCond'] = df['GarageCond'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0})

In [ ]:
print(df['PavedDrive'].isna().sum())
df['PavedDrive'] = LabelEncoder().fit_transform(df['PavedDrive'])

In [ ]:
print(df['WoodDeckSF'].isna().sum())
df['WoodDeckSF'] = StandardScaler().fit_transform(df['WoodDeckSF'].values.reshape(-1,1))

In [ ]:
print(df['OpenPorchSF'].isna().sum())
df['OpenPorchSF'] = StandardScaler().fit_transform(df['OpenPorchSF'].values.reshape(-1,1))

In [ ]:
print(df['EnclosedPorch'].isna().sum())
df['EnclosedPorch'] = StandardScaler().fit_transform(df['EnclosedPorch'].values.reshape(-1,1))

In [ ]:
print(df['3SsnPorch'].isna().sum())
df['3SsnPorch'] = StandardScaler().fit_transform(df['3SsnPorch'].values.reshape(-1,1))

In [ ]:
print(df['ScreenPorch'].isna().sum())
df['ScreenPorch'] = StandardScaler().fit_transform(df['ScreenPorch'].values.reshape(-1,1))

In [ ]:
print(df['PoolArea'].isna().sum())
df['PoolArea'] = StandardScaler().fit_transform(df['PoolArea'].values.reshape(-1,1))

In [ ]:
print(df['PoolQC'].isna().sum())
print(df['PoolQC'].unique())
df['PoolQC'] = df['PoolQC'].replace({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, np.nan: 0})

In [ ]:
print(df['Fence'].isna().sum())
print(df['Fence'].unique())
df['Fence'] = df['Fence'].replace({'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, np.nan: 0})

In [ ]:
print(df['MiscFeature'].isna().sum())
print(df['Fence'].unique())
df['Fence'] = df['Fence'].replace({'Elev': 5, 'Gar2': 4, 'Othr': 3, 'Shed': 2, 'TenC': 1, np.nan: 0})

In [ ]:
print(df['MiscVal'].isna().sum())
df['MiscVal'] = StandardScaler().fit_transform(df['MiscVal'].values.reshape(-1,1))

In [ ]:
print(df['MoSold'].isna().sum())

In [ ]:
print(df['YrSold'].isna().sum())

In [ ]:
print(df['SaleType'].isna().sum())
df['SaleType'] = LabelEncoder().fit_transform(df['SaleType'])

In [ ]:
print(df['SaleCondition'].isna().sum())
df['SaleCondition'] = LabelEncoder().fit_transform(df['SaleCondition'])

In [ ]:
print(df['SalePrice'].isna().sum())
df['MiscVal'] = np.log(df['MiscVal'])